# (실습-5) Vector 유사도 검색 실습 - Elasticsearch

##실습 개요
1) 실습 목적 <br>
  이번 실습에서는 Elasticsearch를 이용하여 vector 유사도 검색을 실험해 본다. <br>
  기존 역색인 검색과 비교하여 결과가 어떻게 달라지는지 확인한다. <br>
2) 수강 목표
  * Elasticsearch를 이용하여 vector 유사도 검색을 할 수 있다.
  * Elasticsearch API를 사용할 수 있다.
  * 기존 역색인 검색 대비 장점을 이해한다.

### 실습 목차
* 1. 환경 설정
* 2. 색인 및 검색

### 데이터셋 개요
* 데이터셋: wikimedia kowiki
* 데이터셋 개요 : wikimedia에서 제공하는 한국어 데이터셋

## 1. 환경 설정


### Elasticsearch 및 관련 패키지 설치
Elasticsearch를 설치한다. <br>
Sentence_transformers 패키지를 설치한다.

In [1]:
# Elasticsearch Python 패키지 설치
!pip install elasticsearch==8.8.0

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 393.8/393.8 kB 2.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 59.9/59.9 kB 6.0 MB/s eta 0:00:00


In [2]:
# Elasticsearch 8.8.0 다운로드 및 압축 풀기

# 리눅스용 엘라스틱서치 서버 설치를 위한 패키지 다운로드
!wget -q https://artifacts.elastic.co/downloads/elasticsearch/elasticsearch-8.8.0-linux-x86_64.tar.gz
!tar -xzf elasticsearch-8.8.0-linux-x86_64.tar.gz

# 코랩 노트북 환경에서 서버 구동을 위해서 PPID 1의 백그라운드 데몬 프로세스가 해당 폴더에 접근이 가능하도록 소유자 변경
!sudo chown -R daemon:daemon elasticsearch-8.8.0/

# 코랩 노트북 환경에서 서버 구동을 위한 리소스 제한/격리를 위해 아래 명령 수행
!umount /sys/fs/cgroup
!apt install cgroup-tools

Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
The following additional packages will be installed:
  libcgroup1
The following NEW packages will be installed:
  cgroup-tools libcgroup1
0 upgraded, 2 newly installed, 0 to remove and 30 not upgraded.
Need to get 121 kB of archives.
After this operation, 435 kB of additional disk space will be used.
Get:1 http://archive.ubuntu.com/ubuntu jammy/universe amd64 libcgroup1 amd64 2.0-2 [49.8 kB]
Get:2 http://archive.ubuntu.com/ubuntu jammy/universe amd64 cgroup-tools amd64 2.0-2 [70.8 kB]
Fetched 121 kB in 1s (146 kB/s)
Selecting previously unselected package libcgroup1:amd64.
(Reading database ... 121658 files and directories currently installed.)
Preparing to unpack .../libcgroup1_2.0-2_amd64.deb ...
Unpacking libcgroup1:amd64 (2.0-2) ...
Selecting previously unselected package cgroup-tools.
Preparing to unpack .../cgroup-tools_2.0-2_amd64.deb ...
Unpacking cgroup-tools (2.0-2) ...
Setting up

In [3]:
# 한글 형태소 분석기 설치
! /content/elasticsearch-8.8.0/bin/elasticsearch-plugin install analysis-nori

-> Installing analysis-nori
-> Downloading analysis-nori from elastic
[=================================================] 100%   
-> Installed analysis-nori
-> Please restart Elasticsearch to activate any plugins installed


In [4]:
# 플러그인 설치 확인 (analysis-nori가 보여야 함)
! /content/elasticsearch-8.8.0/bin/elasticsearch-plugin list

analysis-nori


In [5]:
# 엘라스틱서치의 데몬 인스턴스 만들기
import os
from elasticsearch import Elasticsearch, helpers
import numpy as np
import pandas as pd
import json
from subprocess import Popen, PIPE, STDOUT

es_server = Popen(['elasticsearch-8.8.0/bin/elasticsearch'],
                  stdout=PIPE, stderr=STDOUT,
                  preexec_fn=lambda: os.setuid(1)  # as daemon
                 )

# 인스턴스를 로드하는 데 약간의 시간이 걸림
import time
time.sleep(30)

In [6]:
# 데몬이 구동되었는지 확인 (세개의 daemon process가 있어야 함)
!ps -ef | grep elasticsearch

daemon       950     320 38 05:00 ?        00:00:16 /content/elasticsearch-8.8.0/jdk/bin/java -Xms4m
daemon      1056     950 99 05:01 ?        00:00:49 /content/elasticsearch-8.8.0/jdk/bin/java -Des.n
daemon      1096    1056  0 05:01 ?        00:00:00 /content/elasticsearch-8.8.0/modules/x-pack-ml/p
root        1222     320  0 05:01 ?        00:00:00 /bin/bash -c ps -ef | grep elasticsearch
root        1224    1222  0 05:01 ?        00:00:00 grep elasticsearch


In [7]:
# 데몬 구동후 password 설정 단계 필요
# 명령 실행 후 "Please confirm that you would like to continue"에서 y 입력 필요
!/content/elasticsearch-8.8.0/bin/elasticsearch-setup-passwords auto -url "https://localhost:9200"

******************************************************************************
Note: The 'elasticsearch-setup-passwords' tool has been deprecated. This       command will be removed in a future release.
******************************************************************************

Initiating the setup of passwords for reserved users elastic,apm_system,kibana,kibana_system,logstash_system,beats_system,remote_monitoring_user.
The passwords will be randomly generated and printed to the console.
Please confirm that you would like to continue [y/N]y


Changed password for user apm_system
PASSWORD apm_system = cxXZoior3xCQnStLn0mS

Changed password for user kibana_system
PASSWORD kibana_system = eicE3YUWXU9wmju6f8ug

Changed password for user kibana
PASSWORD kibana = eicE3YUWXU9wmju6f8ug

Changed password for user logstash_system
PASSWORD logstash_system = izR7oOpWoeGA5UxAnmU7

Changed password for user beats_system
PASSWORD beats_system = K6b0lmBpWgwPmjoxnExN

Changed password for user rem

In [8]:
username = 'elastic'

# 위 명령 실행 결과의 마지막 부분인 PASSWORD elastic 값으로 교체 필요
password = 'm2vBuQlFs5kLClraYIvb'

es = Elasticsearch(['https://localhost:9200'], basic_auth=(username, password), ca_certs="/content/elasticsearch-8.8.0/config/certs/http_ca.crt")

resp = dict(es.info())

resp

{'name': '5edf702662ed',
 'cluster_name': 'elasticsearch',
 'cluster_uuid': 'juuhvH41RZ-UV-r43V7sqw',
 'version': {'number': '8.8.0',
  'build_flavor': 'default',
  'build_type': 'tar',
  'build_hash': 'c01029875a091076ed42cdb3a41c10b1a9a5a20f',
  'build_date': '2023-05-23T17:16:07.179039820Z',
  'build_snapshot': False,
  'lucene_version': '9.6.0',
  'minimum_wire_compatibility_version': '7.17.0',
  'minimum_index_compatibility_version': '7.0.0'},
 'tagline': 'You Know, for Search'}

In [9]:
# 임베딩 생성을 위한 벡터 인코더 설치
!pip install sentence-transformers

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 86.0/86.0 kB 1.2 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 9.4 MB/s eta 0:00:00
  Created wheel for sentence-transformers: filename=sentence_transformers-2.2.2-py3-none-any.whl size=125923 sha256=293110eb7bb57c15b67d2f94cdf7ad11a950795981d30b155e400e2ce1794b63
  Stored in directory: /root/.cache/pip/wheels/62/f2/10/1e606fd5f02395388f74e7462910fe851042f97238cbbd902f
Successfully built sentence-transformers


### 데이터 전처리

In [10]:
# 위키미디어로부터 kowiki 데이터를 다운로드 받음
!wget https://dumps.wikimedia.org/kowiki/latest/kowiki-latest-pages-articles1.xml-p1p82407.bz2
# 위키데이터의 노이즈를 제거하고 json 형태로 반환하는 코드를 참조
!git clone https://github.com/attardi/wikiextractor.git
# 다운로드 받은 샘플 위키 데이터를 전처리하여 검색의 입력으로 사용
# 결과는 elastic 폴더에 'extract_result/AA,AB,AC.../wiki_00..99'라는 새로운 폴더에 저장된다.(용량이 비슷하게 나눠서 저장됨)
# 변환결과 wiki_00 파일의 내용 샘플  {"id": "5", "revid": "641228", "url": "https://ko.wikipedia.org/wiki?curid=5", "title": "\uc9c0\...\ud130", "text": "\uc81c\...\ub2e4."}
!python -m wikiextractor.wikiextractor.WikiExtractor kowiki-latest-pages-articles1.xml-p1p82407.bz2 --json -o extract_result

--2024-01-24 05:02:55--  https://dumps.wikimedia.org/kowiki/latest/kowiki-latest-pages-articles1.xml-p1p82407.bz2
Resolving dumps.wikimedia.org (dumps.wikimedia.org)... 208.80.154.142, 2620:0:861:2:208:80:154:142
Connecting to dumps.wikimedia.org (dumps.wikimedia.org)|208.80.154.142|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 74589104 (71M) [application/octet-stream]
Saving to: ‘kowiki-latest-pages-articles1.xml-p1p82407.bz2’

kowiki-latest-pages 100%[===================>]  71.13M  4.15MB/s    in 18s     

2024-01-24 05:03:13 (4.03 MB/s) - ‘kowiki-latest-pages-articles1.xml-p1p82407.bz2’ saved [74589104/74589104]

Cloning into 'wikiextractor'...
remote: Enumerating objects: 771, done.
remote: Counting objects: 100% (30/30), done.
remote: Compressing objects: 100% (16/16), done.
remote: Total 771 (delta 17), reused 21 (delta 14), pack-reused 741
Receiving objects: 100% (771/771), 1.31 MiB | 8.19 MiB/s, done.
Resolving deltas: 100% (450/450), done.
INFO: Pre

In [11]:
import json
from sentence_transformers import SentenceTransformer

# Sentence Transformer 모델 초기화 (한국어 임베딩 생성 가능한 어떤 모델도 가능)
model = SentenceTransformer("hunkim/sentence-transformer-klue")

def get_embedding(sentences):
    # 입력 문장을 인코딩하여 임베딩을 얻음
    return model.encode(sentences)

wiki_dump_json_file = '/content/extract_result/AA/wiki_00'
# 'wiki_dump_json_file'에 있는 JSON 파일 읽어들여 index_docs에 저장
index_docs = []

for line in open(wiki_dump_json_file, encoding="utf-8"):
    # JSON 데이터를 읽어들여 파이썬 딕셔너리로 변환
    json_data = json.loads(line)

    # 'text'에 대한 임베딩을 계산하여 'embeddings' 필드에 추가
    json_data['embeddings_text'] = get_embedding(json_data['text']).tolist()
    json_data['embeddings_title'] = get_embedding(json_data['title']).tolist()

    # 색인할 문서 목록에 추가
    index_docs.append(json_data)

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:88: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


.gitattributes:   0%|          | 0.00/1.35k [00:00<?, ?B/s]

1_Pooling/config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/3.88k [00:00<?, ?B/s]

config.json:   0%|          | 0.00/776 [00:00<?, ?B/s]

...

special_tokens_map.json:   0%|          | 0.00/156 [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/752k [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/618 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/248k [00:00<?, ?B/s]

modules.json:   0%|          | 0.00/229 [00:00<?, ?B/s]

## 2. 색인 및 검색

In [12]:
# Elasticsearch 색인/검색을 위한 공통 함수 정의

from elasticsearch import Elasticsearch, helpers
import json
import pprint as pp

def create_es_index(index, body):
    # 인덱스가 이미 존재하는지 확인
    if es.indices.exists(index=index):
        # 인덱스가 이미 존재하면 설정을 새로운 것으로 갱신하기 위해 삭제
        es.indices.delete(index=index)
    # 지정된 설정으로 새로운 인덱스 생성
    es.indices.create(index=index, body=body)

def delete_es_index(index):
    # 지정된 인덱스 삭제
    es.indices.delete(index=index)

def bulk_add(index, docs):
    # 대량 인덱싱 작업을 준비
    actions = [
        {
            '_index': index,
            '_source': doc
        }
        for doc in docs
    ]
    # Elasticsearch 헬퍼 함수를 사용하여 대량 인덱싱 수행
    return helpers.bulk(es, actions)

def sparse_retrieve(condition, index):
    # 지정된 인덱스에서 생성된 쿼리를 사용하여 검색 수행 (역색인을 이용한 일반 검색)
    return es.search(index=index, body=condition["query_body"], size=condition["size"], sort="_score")

def dense_retrieve(condition, index):
    # 벡터 유사도 검색에 사용할 쿼리 임베딩 가져오기
    query_embedding = get_embedding([condition["query"]])[0]

    # KNN을 사용한 벡터 유사성 검색을 위한 매개변수 설정
    knn = {
        "field": condition["field"],
        "query_vector": query_embedding.tolist(),
        "k": condition["size"],
        "num_candidates": 100
    }

    # 지정된 인덱스에서 벡터 유사도 검색 수행
    return es.search(index=index, knn=knn)


In [13]:
# 색인을 위한 mapping 설정
setting = {
    "settings": {
        "analysis": {
            "analyzer": {
                "nori": {
                    "type": "custom",
                    "tokenizer": "nori_tokenizer",
                    "decompound_mode": "mixed",
                    "filter": ["nori_posfilter"]
                }
            },
            "filter": {
                "nori_posfilter": {
                    "type": "nori_part_of_speech",
                    # 어미, 조사, 구분자, 줄임표, 지정사, 보조 용언 등
                    "stoptags": ["E", "J", "SC", "SE", "SF", "VCN", "VCP", "VX"]
                }
            }
        }
    },
    "mappings": {
        "properties": {
            # 비교 테스트를 위해 meta field를 같이 색인
            "text": {"type": "text", "analyzer": "nori"},
            "title": {"type": "text", "analyzer": "nori"},
            "embeddings_title": {
                "type": "dense_vector",
                "dims": 768,
                "index": True,
                "similarity": "l2_norm"
            },
            "embeddings_text": {
                "type": "dense_vector",
                "dims": 768,
                "index": True,
                "similarity": "l2_norm"
            }
        }
    }
}

In [14]:
# 'setting'으로 설정된 내용으로 'test' 인덱스 생성
create_es_index("test", setting)

# 'test' 인덱스에 대량 색인화 수행
# 각 문서는 'embbedings' 라는 필드를 가짐
ret = bulk_add("test", index_docs)

# 결과 출력
print(ret)

<ipython-input-12-5de2e69d1553>:13: DeprecationWarning: The 'body' parameter is deprecated and will be removed in a future version. Instead use individual parameters.
  es.indices.create(index=index, body=body)


(61, [])


In [15]:
# 역색인을 사용하는 검색 예제
condition_retrieve = {
    "query_body": {
        "query": {
            "match": {
                "text": {
                    "query": "문재인의 친구"
                }
            }
        }
    },
    "size": 5  # Specify the number of documents to retrieve
}
search_result_retrieve = sparse_retrieve(condition_retrieve, "test")

# 결과 출력
for rst in search_result_retrieve['hits']['hits']:
    print('score:', rst['_score'], 'source::', rst['_source'])

<ipython-input-12-5de2e69d1553>:33: DeprecationWarning: The 'body' parameter is deprecated and will be removed in a future version. Instead use individual parameters.
  return es.search(index=index, body=condition["query_body"], size=condition["size"], sort="_score")


score: 4.407809 source:: {'id': '85', 'revid': '368112', 'url': 'https://ko.wikipedia.org/wiki?curid=85', 'title': '노무현', 'text': '노무현(盧武鉉, 1946년 9월 1일~2009년 5월 23일)은 대한민국의 제16대 대통령이다. 판사로 재직 후 부산에서 변호사로 활동하다가 제13·15대 국회의원직을 역임했고, 김대중 정부에서 제6대 해양수산부 장관을 역임했다.\n본관은 광주(光州)이며 경상남도 김해 출생이다. 부산상업고등학교를 졸업하고 막노동에 뛰어들었다가 독학으로 1975년 3월 30세에 제17회 사법시험에 합격하였다. 대전지방법원 판사로 1년을 재직하다가 그만두고 부산에서 변호사 사무실을 개업하여 여러 인권 사건을 변호하였다. 통일민주당 총재 김영삼의 공천을 받아 제13대 총선에 출마하여 부산 동구에서 당선되며 5공비리특별위원으로 활동했다. 1990년 3당 합당에 반대하면서 김영삼과 결별한다. 김대중 정부에서 해양수산부 장관을 지냈고 국민경선제에서 새천년민주당 소속으로 제16대 대선에서 대통령으로 당선되었으나 2003년 말에 새천년민주당을 탈당하고 2004년 초 새천년민주당을 탈당한 개혁 세력들이 주축이 되어 창당한 열린우리당에 입당하였다.\n2004년 무렵 공직선거 및 선거부정방지법이 정한 중립의무 및 헌법 위반을 시유로 야당에 국회로부터 대한민국 헌정 사상 최초로 대통령직 재임 중 탄핵 소추를 당해 대통령 직무가 정지되었다. 하지만 이후 탄핵을 주도했던 새천년민주당과 한나라당, 자유민주연합은 여론의 역풍에 휩싸여 제17대 총선에서 참패하였고 얼마 후 헌법재판소에서 소추안을 기각하며 노무현은 다시 대통령 직무에 복귀하였다.\n주요 업적으로는 권력층에 만연해 있던 권위주의와 정경유착을 타파하고 기존 정권이 하지 못했던 각종 재벌 개혁을 시행한 것이 꼽힌다. 상속증여세의 포괄주의를 도입해 대기업 총수의 탈세 여지를 좁힌 것, 증권 관련 집단소송제를 시행한 것

In [16]:
# 역색인을 사용하는 검색 예제
condition_retrieve = {
    "query_body": {
        "query": {
            "match": {
                "text": {
                    "query": "문재인의 친구",
                    "operator": "AND"
                }
            }
        }
    },
    "size": 5  # Specify the number of documents to retrieve
}
search_result_retrieve = sparse_retrieve(condition_retrieve, "test")

# 결과 출력
for rst in search_result_retrieve['hits']['hits']:
    print('score:', rst['_score'], 'source::', rst['_source'])

score: 4.407809 source:: {'id': '85', 'revid': '368112', 'url': 'https://ko.wikipedia.org/wiki?curid=85', 'title': '노무현', 'text': '노무현(盧武鉉, 1946년 9월 1일~2009년 5월 23일)은 대한민국의 제16대 대통령이다. 판사로 재직 후 부산에서 변호사로 활동하다가 제13·15대 국회의원직을 역임했고, 김대중 정부에서 제6대 해양수산부 장관을 역임했다.\n본관은 광주(光州)이며 경상남도 김해 출생이다. 부산상업고등학교를 졸업하고 막노동에 뛰어들었다가 독학으로 1975년 3월 30세에 제17회 사법시험에 합격하였다. 대전지방법원 판사로 1년을 재직하다가 그만두고 부산에서 변호사 사무실을 개업하여 여러 인권 사건을 변호하였다. 통일민주당 총재 김영삼의 공천을 받아 제13대 총선에 출마하여 부산 동구에서 당선되며 5공비리특별위원으로 활동했다. 1990년 3당 합당에 반대하면서 김영삼과 결별한다. 김대중 정부에서 해양수산부 장관을 지냈고 국민경선제에서 새천년민주당 소속으로 제16대 대선에서 대통령으로 당선되었으나 2003년 말에 새천년민주당을 탈당하고 2004년 초 새천년민주당을 탈당한 개혁 세력들이 주축이 되어 창당한 열린우리당에 입당하였다.\n2004년 무렵 공직선거 및 선거부정방지법이 정한 중립의무 및 헌법 위반을 시유로 야당에 국회로부터 대한민국 헌정 사상 최초로 대통령직 재임 중 탄핵 소추를 당해 대통령 직무가 정지되었다. 하지만 이후 탄핵을 주도했던 새천년민주당과 한나라당, 자유민주연합은 여론의 역풍에 휩싸여 제17대 총선에서 참패하였고 얼마 후 헌법재판소에서 소추안을 기각하며 노무현은 다시 대통령 직무에 복귀하였다.\n주요 업적으로는 권력층에 만연해 있던 권위주의와 정경유착을 타파하고 기존 정권이 하지 못했던 각종 재벌 개혁을 시행한 것이 꼽힌다. 상속증여세의 포괄주의를 도입해 대기업 총수의 탈세 여지를 좁힌 것, 증권 관련 집단소송제를 시행한 것

<ipython-input-12-5de2e69d1553>:33: DeprecationWarning: The 'body' parameter is deprecated and will be removed in a future version. Instead use individual parameters.
  return es.search(index=index, body=condition["query_body"], size=condition["size"], sort="_score")


In [17]:
# Vector 유사도 사용한 검색 예제
condition_retrieve = {
    "field": "embeddings_text",
    "query": "문재인의 친구",
    "size": 5  # Specify the number of documents to retrieve
}
search_result_retrieve = dense_retrieve(condition_retrieve, "test")

# 결과 출력
for rst in search_result_retrieve['hits']['hits']:
    print('score:', rst['_score'], 'source::', rst['_source'])

score: 0.0061816927 source:: {'id': '85', 'revid': '368112', 'url': 'https://ko.wikipedia.org/wiki?curid=85', 'title': '노무현', 'text': '노무현(盧武鉉, 1946년 9월 1일~2009년 5월 23일)은 대한민국의 제16대 대통령이다. 판사로 재직 후 부산에서 변호사로 활동하다가 제13·15대 국회의원직을 역임했고, 김대중 정부에서 제6대 해양수산부 장관을 역임했다.\n본관은 광주(光州)이며 경상남도 김해 출생이다. 부산상업고등학교를 졸업하고 막노동에 뛰어들었다가 독학으로 1975년 3월 30세에 제17회 사법시험에 합격하였다. 대전지방법원 판사로 1년을 재직하다가 그만두고 부산에서 변호사 사무실을 개업하여 여러 인권 사건을 변호하였다. 통일민주당 총재 김영삼의 공천을 받아 제13대 총선에 출마하여 부산 동구에서 당선되며 5공비리특별위원으로 활동했다. 1990년 3당 합당에 반대하면서 김영삼과 결별한다. 김대중 정부에서 해양수산부 장관을 지냈고 국민경선제에서 새천년민주당 소속으로 제16대 대선에서 대통령으로 당선되었으나 2003년 말에 새천년민주당을 탈당하고 2004년 초 새천년민주당을 탈당한 개혁 세력들이 주축이 되어 창당한 열린우리당에 입당하였다.\n2004년 무렵 공직선거 및 선거부정방지법이 정한 중립의무 및 헌법 위반을 시유로 야당에 국회로부터 대한민국 헌정 사상 최초로 대통령직 재임 중 탄핵 소추를 당해 대통령 직무가 정지되었다. 하지만 이후 탄핵을 주도했던 새천년민주당과 한나라당, 자유민주연합은 여론의 역풍에 휩싸여 제17대 총선에서 참패하였고 얼마 후 헌법재판소에서 소추안을 기각하며 노무현은 다시 대통령 직무에 복귀하였다.\n주요 업적으로는 권력층에 만연해 있던 권위주의와 정경유착을 타파하고 기존 정권이 하지 못했던 각종 재벌 개혁을 시행한 것이 꼽힌다. 상속증여세의 포괄주의를 도입해 대기업 총수의 탈세 여지를 좁힌 것, 증권 관련 집단소송제를 시

In [18]:
# 역색인을 사용하는 검색 예제 - title field 사용
condition_retrieve = {
    "query_body": {
        "query": {
            "match": {
                "title": {
                    "query": "문재인의 친구"
                }
            }
        }
    },
    "size": 5  # Specify the number of documents to retrieve
}
search_result_retrieve = sparse_retrieve(condition_retrieve, "test")

# 결과 출력
for rst in search_result_retrieve['hits']['hits']:
    print('score:', rst['_score'], 'source::', rst['_source'])

<ipython-input-12-5de2e69d1553>:33: DeprecationWarning: The 'body' parameter is deprecated and will be removed in a future version. Instead use individual parameters.
  return es.search(index=index, body=condition["query_body"], size=condition["size"], sort="_score")


In [19]:
# Vector 유사도 사용한 검색 예제 - title field 사용
condition_retrieve = {
    "query": "문재인의 친구",
    "field": "embeddings_title",
    "size": 5  # Specify the number of documents to retrieve
}
search_result_retrieve = dense_retrieve(condition_retrieve, "test")

# 결과 출력
for rst in search_result_retrieve['hits']['hits']:
    print('score:', rst['_score'], 'source::', rst['_source'])

score: 0.0073537673 source:: {'id': '85', 'revid': '368112', 'url': 'https://ko.wikipedia.org/wiki?curid=85', 'title': '노무현', 'text': '노무현(盧武鉉, 1946년 9월 1일~2009년 5월 23일)은 대한민국의 제16대 대통령이다. 판사로 재직 후 부산에서 변호사로 활동하다가 제13·15대 국회의원직을 역임했고, 김대중 정부에서 제6대 해양수산부 장관을 역임했다.\n본관은 광주(光州)이며 경상남도 김해 출생이다. 부산상업고등학교를 졸업하고 막노동에 뛰어들었다가 독학으로 1975년 3월 30세에 제17회 사법시험에 합격하였다. 대전지방법원 판사로 1년을 재직하다가 그만두고 부산에서 변호사 사무실을 개업하여 여러 인권 사건을 변호하였다. 통일민주당 총재 김영삼의 공천을 받아 제13대 총선에 출마하여 부산 동구에서 당선되며 5공비리특별위원으로 활동했다. 1990년 3당 합당에 반대하면서 김영삼과 결별한다. 김대중 정부에서 해양수산부 장관을 지냈고 국민경선제에서 새천년민주당 소속으로 제16대 대선에서 대통령으로 당선되었으나 2003년 말에 새천년민주당을 탈당하고 2004년 초 새천년민주당을 탈당한 개혁 세력들이 주축이 되어 창당한 열린우리당에 입당하였다.\n2004년 무렵 공직선거 및 선거부정방지법이 정한 중립의무 및 헌법 위반을 시유로 야당에 국회로부터 대한민국 헌정 사상 최초로 대통령직 재임 중 탄핵 소추를 당해 대통령 직무가 정지되었다. 하지만 이후 탄핵을 주도했던 새천년민주당과 한나라당, 자유민주연합은 여론의 역풍에 휩싸여 제17대 총선에서 참패하였고 얼마 후 헌법재판소에서 소추안을 기각하며 노무현은 다시 대통령 직무에 복귀하였다.\n주요 업적으로는 권력층에 만연해 있던 권위주의와 정경유착을 타파하고 기존 정권이 하지 못했던 각종 재벌 개혁을 시행한 것이 꼽힌다. 상속증여세의 포괄주의를 도입해 대기업 총수의 탈세 여지를 좁힌 것, 증권 관련 집단소송제를 시

In [20]:
# 역색인을 사용하는 검색 예제
condition_retrieve = {
    "query_body": {
        "query": {
            "match": {
                "title": {
                    "query": "우리나라 열 여섯번째 대통령이 누구야?"
                }
            }
        }
    },
    "size": 5  # Specify the number of documents to retrieve
}
search_result_retrieve = sparse_retrieve(condition_retrieve, "test")

# 결과 출력
for rst in search_result_retrieve['hits']['hits']:
    print('score:', rst['_score'], 'source::', rst['_source'])

score: 3.3614042 source:: {'id': '20', 'revid': '106455', 'url': 'https://ko.wikipedia.org/wiki?curid=20', 'title': '나라 목록', 'text': '이 목록에 실린 국가 기준은 1933년 몬테비데오 협약 1장을 참고로 하였다. 협정에 따르면, 국가는 다음의 조건을 만족해야 한다.\n특히, 마지막 조건은 국제 공동체의 참여 용인을 내포하고 있기 때문에, 다른 나라의 승인이 매우 중요한 역할을 할 수 있다. 이 목록에 포함된 모든 국가는 보통 이 기준을 만족하는 것으로 보이는 자주적이고 독립적인 국가이다. 하지만 몬테비데오 협약 기준을 만족하는지의 여부는 많은 국가가 논쟁이 되고 있는 실정이다. 또한, 몬테비데오 협약 기준만이 국가 지위의 충분한 자격이든 아니든, 국제법의 견해 차이는 존재할 수 있다. 이 물음에 대한 다른 이론에 대한 고리는 아래에서 볼 수 있다.\n기준.\n위 기준에 논거하여 이 목록은 다음 208개 국가를 포함하고 있다.\n미승인 국가.\n이 목록은 주권을 주장하고 점유한 영토를 실제로 관리하고 있으나, 많은 국가와 외교관계를 맺지 못한 나라를 설명하고 있다. 마이크로네이션는 이 목록에 포함하지 않는다.', 'embeddings_text': [-0.10890917479991913, -0.2800323963165283, -0.036791592836380005, -0.258735328912735, -0.23113508522510529, -0.48579585552215576, -0.0031200258526951075, -0.27645811438560486, -0.1901565045118332, -0.3603999614715576, 0.19388224184513092, 0.3062269687652588, 0.13646602630615234, -0.08949089050292969, -0.4891112744808197, 0.24047450721263885, -0.

<ipython-input-12-5de2e69d1553>:33: DeprecationWarning: The 'body' parameter is deprecated and will be removed in a future version. Instead use individual parameters.
  return es.search(index=index, body=condition["query_body"], size=condition["size"], sort="_score")


In [21]:
# Vector 유사도 사용한 검색 예제
condition_retrieve = {
    "query": "우리나라 열 여섯번째 대통령이 누구야?",
    "field": "embeddings_title",
    "size": 5  # Specify the number of documents to retrieve
}
search_result_retrieve = dense_retrieve(condition_retrieve, "test")

# 결과 출력
for rst in search_result_retrieve['hits']['hits']:
    print('score:', rst['_score'], 'source::', rst['_source'])

score: 0.006525172 source:: {'id': '34', 'revid': '36250640', 'url': 'https://ko.wikipedia.org/wiki?curid=34', 'title': '대한민국 제16대 대통령 선거', 'text': '대한민국 제16대 대통령 선거는 2002년 12월 19일 목요일 치뤄진 대통령 선거로, 21세기에 처음으로 치뤄진 대한민국 대통령 선거이다. 제15대 김대중 대통령의 차기 대통령을 뽑기 위한 선거이다.\n16대 대선은 지난 15대 대선에서 간발의 차로 낙선하고 재도전한 이회창 한나라당 후보와 사상 최초의 국민 참여 경선을 통해 여당의 대통령 후보가 된 해양수산부 장관 출신 노무현 새천년민주당 후보의 양강 구도로 진행되었다.\n대선 재수생인 이회창 후보는 경험이나 세력 면에서 노무현 후보보다 대권 고지에 좀 더 유리할 것으로 점쳐졌으나, 이전 대선부터 불거진 이회창 후보의 두 아들의 병역기피 논란, 노사모를 비롯한 네티즌들의 열성적인 노무현 지지, 정몽준 후보와의 단일화 성공 등에 힘입어 노무현 후보가 당선되었다.\n선거 정보.\n선거권.\n만 20세 이상의 대한민국 국민은 선거권이 있었다. 즉, 1982년 12월 19일 이전에 태어난 사람은 투표를 할 자격이 있었다.\n피선거권.\n만 40세 이상의 대한민국 국민은 피선거권을 가졌다. 즉, 1962년 12월 19일 이전에 태어난 사람은 후보자가 될 자격이 있었다.\n후보.\n새천년민주당.\n새천년민주당은 3월 9일부터 4월 27일까지 한국 정당 역사상 최초로 국민 참여 경선을 실시하고 과반 득표자인 노무현 전, 판사, 변호사, 해양수산부 장관을 대통령 후보로 선출하였다.\n한나라당.\n한나라당은 4월 13일부터 5월 9일까지 국민 참여 경선을 실시하고 최다 득표자인 이회창 전 당 총재를 대통령 후보로 선출하였다.\n민주노동당.\n민주노동당은 9월 8일 당원들에 의한 단일 후보 찬반 투표를 통해 권영길 당 대표를 대통령 후보로 선출하였다.\n국민통합2

In [22]:
# 역색인을 사용하는 검색 예제 - 오탈자
condition_retrieve = {
    "query_body": {
        "query": {
            "match": {
                "title": {
                    "query": "우리나라 열 여섯번째 대통려이 누구야?"
                }
            }
        }
    },
    "size": 5  # Specify the number of documents to retrieve
}
search_result_retrieve = sparse_retrieve(condition_retrieve, "test")

# 결과 출력
for rst in search_result_retrieve['hits']['hits']:
    print('score:', rst['_score'], 'source::', rst['_source'])

score: 3.3614042 source:: {'id': '20', 'revid': '106455', 'url': 'https://ko.wikipedia.org/wiki?curid=20', 'title': '나라 목록', 'text': '이 목록에 실린 국가 기준은 1933년 몬테비데오 협약 1장을 참고로 하였다. 협정에 따르면, 국가는 다음의 조건을 만족해야 한다.\n특히, 마지막 조건은 국제 공동체의 참여 용인을 내포하고 있기 때문에, 다른 나라의 승인이 매우 중요한 역할을 할 수 있다. 이 목록에 포함된 모든 국가는 보통 이 기준을 만족하는 것으로 보이는 자주적이고 독립적인 국가이다. 하지만 몬테비데오 협약 기준을 만족하는지의 여부는 많은 국가가 논쟁이 되고 있는 실정이다. 또한, 몬테비데오 협약 기준만이 국가 지위의 충분한 자격이든 아니든, 국제법의 견해 차이는 존재할 수 있다. 이 물음에 대한 다른 이론에 대한 고리는 아래에서 볼 수 있다.\n기준.\n위 기준에 논거하여 이 목록은 다음 208개 국가를 포함하고 있다.\n미승인 국가.\n이 목록은 주권을 주장하고 점유한 영토를 실제로 관리하고 있으나, 많은 국가와 외교관계를 맺지 못한 나라를 설명하고 있다. 마이크로네이션는 이 목록에 포함하지 않는다.', 'embeddings_text': [-0.10890917479991913, -0.2800323963165283, -0.036791592836380005, -0.258735328912735, -0.23113508522510529, -0.48579585552215576, -0.0031200258526951075, -0.27645811438560486, -0.1901565045118332, -0.3603999614715576, 0.19388224184513092, 0.3062269687652588, 0.13646602630615234, -0.08949089050292969, -0.4891112744808197, 0.24047450721263885, -0.

<ipython-input-12-5de2e69d1553>:33: DeprecationWarning: The 'body' parameter is deprecated and will be removed in a future version. Instead use individual parameters.
  return es.search(index=index, body=condition["query_body"], size=condition["size"], sort="_score")


In [23]:
# Vector 유사도 사용한 검색 예제 - 오탈자
condition_retrieve = {
    "query": "우리나라 16대 대통려이 누구야?",
    "field": "embeddings_title",
    "size": 5  # Specify the number of documents to retrieve
}
search_result_retrieve = dense_retrieve(condition_retrieve, "test")

# 결과 출력
for rst in search_result_retrieve['hits']['hits']:
    print('score:', rst['_score'], 'source::', rst['_source'])

score: 0.01131315 source:: {'id': '34', 'revid': '36250640', 'url': 'https://ko.wikipedia.org/wiki?curid=34', 'title': '대한민국 제16대 대통령 선거', 'text': '대한민국 제16대 대통령 선거는 2002년 12월 19일 목요일 치뤄진 대통령 선거로, 21세기에 처음으로 치뤄진 대한민국 대통령 선거이다. 제15대 김대중 대통령의 차기 대통령을 뽑기 위한 선거이다.\n16대 대선은 지난 15대 대선에서 간발의 차로 낙선하고 재도전한 이회창 한나라당 후보와 사상 최초의 국민 참여 경선을 통해 여당의 대통령 후보가 된 해양수산부 장관 출신 노무현 새천년민주당 후보의 양강 구도로 진행되었다.\n대선 재수생인 이회창 후보는 경험이나 세력 면에서 노무현 후보보다 대권 고지에 좀 더 유리할 것으로 점쳐졌으나, 이전 대선부터 불거진 이회창 후보의 두 아들의 병역기피 논란, 노사모를 비롯한 네티즌들의 열성적인 노무현 지지, 정몽준 후보와의 단일화 성공 등에 힘입어 노무현 후보가 당선되었다.\n선거 정보.\n선거권.\n만 20세 이상의 대한민국 국민은 선거권이 있었다. 즉, 1982년 12월 19일 이전에 태어난 사람은 투표를 할 자격이 있었다.\n피선거권.\n만 40세 이상의 대한민국 국민은 피선거권을 가졌다. 즉, 1962년 12월 19일 이전에 태어난 사람은 후보자가 될 자격이 있었다.\n후보.\n새천년민주당.\n새천년민주당은 3월 9일부터 4월 27일까지 한국 정당 역사상 최초로 국민 참여 경선을 실시하고 과반 득표자인 노무현 전, 판사, 변호사, 해양수산부 장관을 대통령 후보로 선출하였다.\n한나라당.\n한나라당은 4월 13일부터 5월 9일까지 국민 참여 경선을 실시하고 최다 득표자인 이회창 전 당 총재를 대통령 후보로 선출하였다.\n민주노동당.\n민주노동당은 9월 8일 당원들에 의한 단일 후보 찬반 투표를 통해 권영길 당 대표를 대통령 후보로 선출하였다.\n국민통합21

#Reference

## Required Package

sentence_transformers=2.2.2 <br>
elasticsearch==8.8.0
